In [1]:
using NCDatasets
using DIVAnd
using PyPlot
using Dates
using Statistics
using Random
using Printf

## Files and directories

In [2]:
varname = "Salinity"
outputdir = "/home/ctroupin/dox/Projects/SeaDataCloud/NorthSea/"
figdir = "./NorthSea/figures/Profiles/"
varname = "Salinity"
annualS = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_NorthSea.4Danl_annual.nc")
seasonalS = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_NorthSea.4Danl_seasonal.nc")
monthlyS = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_NorthSea.4Danl_monthly.nc")
if isfile(annualS) & isfile(seasonalS) & isfile(monthlyS)
    @info "All the files exist"
else
    @warn "Some files don't exist, check paths and file names"
end

varname = "Temperature"
annualT = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_NorthSea.4Danl_annual.nc")
seasonalT = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_NorthSea.4Danl_seasonal.nc")
monthlyT = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_NorthSea.4Danl_monthly.nc")
if isfile(annualT) & isfile(seasonalT) & isfile(monthlyT)
    @info "All the files exist"
else
    @warn "Some files don't exist, check paths and file names"
end

┌ Info: All the files exist
└ @ Main In[2]:9
┌ Info: All the files exist
└ @ Main In[2]:19


In [3]:
function read_field_nc(datafile::String, varname::String)
    local lon, lat, time, depth, field
    Dataset(datafile) do ds
        time = ds["time"][:]
        lon = ds["lon"][:]
        lat = ds["lat"][:]
        depth = ds["depth"][:]
        field = ds[varname][:]
    end
    return time, lon, lat, depth, field
end

read_field_nc (generic function with 1 method)

In [4]:
function get_mean_profile(field::Array, depth::Array)
    local meanprof = zeros(length(depth))
    for ii=1:length(depth)
        meanprof[ii] = mean(skipmissing(field[:,:,ii,1]));
    end
    return meanprof
end

get_mean_profile (generic function with 1 method)

In [5]:
function plot_mean_profile(field::Array, depth::Array, varname::String, 
        figname::String="", figtitle::String="")
    PyPlot.figure()
    PyPlot.plot(field, depth, "ko-")
    PyPlot.gca().invert_yaxis()
    xlabel(varname)
    ylabel("Depth (m)")
    
    if figtitle !== ""
       PyPlot.title(figtitle) 
    end
    if figname !== ""
        PyPlot.savefig(figname, dpi=300, bbox_inches="tight")
        PyPlot.close_figs()
    end
end

plot_mean_profile (generic function with 3 methods)

## Plots

In [6]:
seasons = ["winter", "spring", "summer", "fall"]
months = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"];

### Annual salinity

In [7]:
time, lon, lat, depth, S = read_field_nc(annualS, "Salinity");
meanS = get_mean_profile(S, depth);
plot_mean_profile(meanS, depth, "Salinity", joinpath(figdir, "salinity_annual_mean.png"), "Annual mean")

### Annual temperature

In [8]:
time, lon, lat, depth, T = read_field_nc(annualT, "Temperature");
meanT = get_mean_profile(T, depth);
plot_mean_profile(meanT, depth, "Temperature (°C)", joinpath(figdir, "temperature_annual_mean.png"), "Annual mean")

### Seasonal salinity

In [9]:
time, lon, lat, depth, S = read_field_nc(seasonalS, "Salinity");
for (i, s) in enumerate(seasons)
    ii = @sprintf("%02d", i)
    meanS = get_mean_profile(S[:,:,:,i], depth);
    plot_mean_profile(meanS, depth, "Salinity", joinpath(figdir, "salinity_seasonal_mean_$(ii).png"), 
        "Seasonal mean: $(s)")
end

### Seasonal temperature

In [10]:
time, lon, lat, depth, T = read_field_nc(seasonalT, "Temperature");
for (i, s) in enumerate(seasons)
    ii = @sprintf("%02d", i)
    meanT = get_mean_profile(T[:,:,:,i], depth);
    plot_mean_profile(meanT, depth, "Temperature (°C)", joinpath(figdir, "temperature_seasonal_mean_$(ii).png"), 
        "Seasonal mean: $(s)")
end

### Monthly salinity

In [11]:
time, lon, lat, depth, S = read_field_nc(monthlyS, "Salinity");
for (i, m) in enumerate(months)
    ii = @sprintf("%02d", i)
    @debug("Working on month: $(m)")
    meanS = get_mean_profile(S[:,:,:,i], depth);
    plot_mean_profile(meanS, depth, "Salinity", joinpath(figdir, "salinity_monthly_mean_$(ii).png"), 
        "Monthly mean: $(m)")
end

### Monthly temperature

In [12]:
time, lon, lat, depth, T = read_field_nc(monthlyT, "Temperature");
for (i, m) in enumerate(months)
    ii = @sprintf("%02d", i)
    @debug("Working on month: $(m)")
    meanT = get_mean_profile(T[:,:,:,i], depth);
    plot_mean_profile(meanT, depth, "Temperature (°C)", joinpath(figdir, "temperature_monthly_mean_$(ii).png"), 
        "Monthly mean: $(m)")
end